# Second Quantized Hamiltonian

In [1]:
import time

import numpy as np
from openfermion import FermionOperator

from classiq import *
from classiq.applications.chemistry.mapping import FermionToQubitMapper
from classiq.applications.chemistry.op_utils import qubit_op_to_pauli_terms
from classiq.applications.chemistry.problems import FermionHamiltonianProblem

In [2]:
op_list = [
    FermionOperator("0^ 0", 0.2),
    FermionOperator("0^ 1^ 1 0", -0.1),
    FermionOperator("2^ 3^ 2 3", -0.3),
]
hamiltonian = sum(op_list)

ham_problem = FermionHamiltonianProblem(
    fermion_hamiltonian=hamiltonian, n_particles=(1, 1)
)

print(ham_problem.fermion_hamiltonian)

0.2 [0^ 0] +
-0.1 [0^ 1^ 1 0] +
-0.3 [2^ 3^ 2 3]


In [3]:
mapper = FermionToQubitMapper()
vqe_hamiltonian = qubit_op_to_pauli_terms(mapper.map(ham_problem.fermion_hamiltonian))
num_qubits = mapper.get_num_qubits(ham_problem)

reps = 3
num_params = reps * num_qubits


@qfunc
def main(params: CArray[CReal, num_params], state: Output[QArray[QBit, num_qubits]]):
    allocate(state)
    full_hea(
        num_qubits=num_qubits,
        operands_1qubit=[lambda _, q: X(q), lambda theta, q: RY(theta, q)],
        operands_2qubit=[lambda _, q1, q2: CX(q1, q2)],
        is_parametrized=[0, 1, 0],
        angle_params=params,
        connectivity_map=[(0, 1), (1, 2), (2, 3)],
        reps=reps,
        x=state,
    )


write_qmod(main, "second_quantized_hamiltonian")

In [4]:
qprog = synthesize(main)
show(qprog)

Quantum program link: https://nightly.platform.classiq.io/circuit/30VHtUc0wpanVWbQj790uE9lAWP


In [5]:
with ExecutionSession(qprog) as es:
    result = es.minimize(
        cost_function=vqe_hamiltonian,
        initial_params={"params": [0.0] * num_params},
        max_iteration=200,
    )

In [6]:
optimizer_res = result[-1][0]
print("vqe result:", optimizer_res)

vqe result: 0.000488281250000014
